# Script para buscar o banco de redações da UOL
Este script busca todas as redações do banco de redações da UOL.

In [171]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import re
from collections import defaultdict

soup = BeautifulSoup(open('../datasets/Banco_Redacoes_vFull.html'), 'html.parser')

months = soup.find_all('div', class_='thumbnails-wrapper')
links = [month.contents[0]['href'] for month in months]
len(links)

55

In [172]:
redacoes = []
for link in tqdm(links):
    html = requests.get(link).content
    data = BeautifulSoup(html, 'html.parser')
    search_results = data.find_all('div', class_='rt-line-option')
    search_links = [result.contents[0]['href'] for result in search_results]
    redacoes += search_links

len(redacoes)


  7%|▋         | 4/55 [00:12<03:05,  3.64s/it]

KeyboardInterrupt: 

In [170]:
import csv

for redacao in tqdm(redacoes):
    redacao_data = []
    html = requests.get(redacao).content
    data = BeautifulSoup(html, 'html.parser')
    search_results = data.find_all('div', class_='container-composition')
    search_evaluation = data.find_all('div', class_='rt-line-option')
    redacao_title = search_results[0].contents[0].string
    redacao_classes = [contents.contents[2].string for contents in search_evaluation if contents.contents[0].string]
    redacao_texto = search_results[0].find('div', class_='text-composition')
    redacao_texto = re.sub('<span style="color:#00b050">[a-z,\sA-ZáéíóúÁÉÍÓÚàÀçõãÇâêÃÕ]+<\/span>', ' ', str(redacao_texto))
    redacao_texto = re.sub('<[^>]*>', ' ', redacao_texto)
    redacao_texto = re.sub('\"', '', redacao_texto)
    redacao_texto = re.sub('\s+', ' ', redacao_texto)
    redacao_texto = re.sub('\s\.', '.', redacao_texto)
    redacao_texto = re.sub('\s,', ',', redacao_texto)
    redacao_texto.lower()
    redacao_texto = redacao_texto.strip()

    redacao_data.append([redacao_title] + [redacao_texto] + redacao_classes)

    with open("output.csv", "a") as f:
        writer = csv.writer(f)
        writer.writerows(redacao_data)


  0%|          | 3/854 [00:04<18:02,  1.27s/it]

KeyboardInterrupt: 

In [166]:
redacao_data

['Qualificações para o mercado de trabalho',
 'Anos após anos uma vaga de emprego demandar cada vez mais qualificações do empregado, que na maioria das vezes concluíram o curso superior e são incapazes de trabalha nesse ramo e optam por um emprego informal. Esse fenômeno ocorre porque o mercado de trabalho se modifica constantemente de maneira que algumas profissões são extintas pois são ocupadas por maquinas. No filme Estrelas além do Tempo, de 2016, contar a história de três calculadoras humanas, que eram muito requisitadas por centros de pesquisa antes dos surgimentos das modernas calculadoras eletrônicas e dos computadores. No longa-metragem, vemos a personagem Dorothy Vougham que ao saber que a Nasa tinha comprado um novo computador que fazia 24mil cálculos por segundo, o que colocava seu emprego em risco, Dorothy resolveu aprender a programar esse mesmo computador e ensinar suas companheiras. No mercado de trabalho e necessário está sempre se adaptando pois ele e instável fazendo

In [70]:
redacao_texto = search_results[0].find('div', class_='text-composition')
redacao_texto = re.sub('<span style="color:#00b050">[a-z,\sA-ZáéíóúÁÉÍÓÚàÀçõã]+<\/span>', '', str(redacao_texto))
redacao_texto = re.sub('<[^>]*>', '', redacao_texto)
redacao_texto

'Anos após anos uma vaga de emprego demandar cada vez mais qualificações do empregado, que na maioria das vezes concluíram o curso superior e são incapazes de trabalha nesse ramo e optam por um emprego informal. Esse fenômeno ocorre porque o mercado de trabalho se modifica constantemente de maneira que algumas profissões são extintas pois são ocupadas por maquinas .No filme "Estrelas além do Tempo", de 2016, contar a história de três calculadoras humanas , que eram muito requisitadas por centros de pesquisa antes dos surgimentos das modernas calculadoras eletrônicas e dos computadores. No longa-metragem, vemos a personagem Dorothy Vougham que ao saber que a Nasa tinha comprado um novo computador que fazia 24mil cálculos por segundo, o que colocava seu emprego em risco, Dorothy resolveu aprender a programar esse mesmo computador e ensinar suas companheiras.No mercado de trabalho e necessário está sempre se adaptando pois ele e instável fazendo com que algumas profissões sumam enquanto n

1046

In [42]:
redacao_classes = [(contents.contents[0].string, contents.contents[2].string) for contents in search_evaluation if contents.contents[0].string]
redacao_classes

[('Demonstrar domínio da norma culta da língua escrita.', '80'),
 ('Compreender a proposta da redação e aplicar conceito das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo.',
  '120'),
 ('Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.',
  '120'),
 ('Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação.',
  '80'),
 ('Elaborar a proposta de solução para o problema abordado, mostrando respeito aos valores humanos e considerando a diversidade sociocultural.',
  '80'),
 ('Nota final', '480')]